In [2]:
topic_file = "/data/movie_data/movies.csv"
users_file = "/data/movie_data/users.csv"
ratings_file = "/data/movie_data/rating.csv"

In [3]:
import pandas as pd
topics = pd.read_csv(topic_file)

In [4]:
topics.head()

,ID,NAME,CATEGORY
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings = pd.read_csv(ratings_file)
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
users = pd.read_csv(users_file)
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
user_rating = pd.merge(users,ratings,on='UserID')
user_rating_mv = pd.merge(user_rating, topics, left_on="MovieID", right_on="ID") 

In [8]:
user_rating_mv

,UserID,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,ID,NAME,CATEGORY
0,1,F,1,10,48067,1193,5,978300760,1193,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,1193,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,1193,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,1193,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,1193,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,M,18,17,47901,2198,5,958846401,2198,Modulations (1998),Documentary
1000205,5675,M,35,14,30030,2703,3,976029116,2703,Broken Vessels (1998),Drama
1000206,5780,M,18,17,92886,2845,1,958153068,2845,White Boys (1999),Drama
1000207,5851,F,18,20,55410,3607,5,957756608,3607,One Little Indian (1973),Comedy|Drama|Western


In [58]:
#df = pd.DataFrame().reset_index()

top_movies = user_rating_mv[['UserID','Gender','Age', 'NAME']].groupby(['Gender','Age','NAME']).count().reset_index()

In [69]:
def top_movies(age, gender):
    print(age)
    print(gender)
    top_movies

In [70]:
top_movies('M','12')

M
12


In [57]:
print("Hi")

Hi
